In [1]:
import pickle
from collections import OrderedDict
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
from periodictable import elements
from tqdm import tqdm

In [2]:
from jitr.optical_potentials import kduq, wlh

In [3]:
import rxmc

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [4]:
import elm

In [5]:
neutron = (1, 0)
proton = (1, 1)

In [14]:
with open(Path("../prior/prior_distribution.pickle"), "rb") as f:
    prior_distribution = pickle.load(f)
param_samples = rxmc.params.array_to_list(prior_distribution.rvs(1000), [p.name for p in elm.params])

In [15]:
param_samples[0]

OrderedDict([('V0', np.float64(56.92343461476179)),
             ('W0', np.float64(14.772410903638328)),
             ('Wd0', np.float64(2.048836983332338)),
             ('V1', np.float64(6.253158373254973)),
             ('W1', np.float64(2.4901245716855356)),
             ('Wd1', np.float64(29.003945161217835)),
             ('alpha', np.float64(-0.4320259825764513)),
             ('gamma_w', np.float64(55.177005146181386)),
             ('gamma_d', np.float64(52.3856039722908)),
             ('r0A', np.float64(1.18546013236974)),
             ('r1A', np.float64(1.1980466952505506)),
             ('a0', np.float64(0.6821556279538155)),
             ('a1', np.float64(0.7096455579233042))])

## read in measurements and pre-computed workspaces

In [16]:
corpus_dir = Path("./corpus/")
output_dir = Path("./elm_prior_propagation/")
output_dir.mkdir(parents=True, exist_ok=True)

In [17]:
with open(corpus_dir / "nn_elastic_data.pkl", "rb") as f:
    nn_elastic_data = pickle.load(f)
with open(corpus_dir / "pp_elastic_data.pkl", "rb") as f:
    pp_elastic_data = pickle.load(f)

In [18]:
with open(corpus_dir / "nn_elastic_workspaces.pkl", "rb") as f:
    nn_elastic_workspaces = pickle.load(f)
with open(corpus_dir / "pp_elastic_workspaces.pkl", "rb") as f:
    pp_elastic_workspaces = pickle.load(f)

## set up corpora from workspaces

In [19]:
nn_corpus = rxmc.reaction_corpus.ElasticAngularCorpus(
    elm.elm.calculate_elastic_differential_xs,
    [p.name for p in elm.params],
    "ELM",
    "(n,n)",
    "dXS/dA",
    nn_elastic_workspaces,
    nn_elastic_data,
)
pp_corpus = rxmc.reaction_corpus.ElasticAngularCorpus(
    elm.elm.calculate_elastic_differential_xs,
    [p.name for p in elm.params],
    "ELM",
    "(p,p)",
    "dXS/dRuth",
    pp_elastic_workspaces,
    pp_elastic_data,
)
with open(output_dir / "elm_nn_corpus.pkl", "wb") as f:
    pickle.dump(nn_corpus, f)
with open(output_dir / "elm_pp_corpus.pkl", "wb") as f:
    pickle.dump(pp_corpus, f)

## Propagate ELM prior into constraint observables

In [20]:
yth_nn = []
for i, constraint in enumerate(tqdm(nn_corpus.constraints)):
    yth_nn.append(np.vstack([constraint.model(p) for p in param_samples]))

100%|███████████████████████████████████████████████████████████████| 65/65 [02:24<00:00,  2.22s/it]


In [ ]:
yth_pp = []
for i, constraint in enumerate(tqdm(pp_corpus.constraints)):
    yth_pp.append(np.vstack([constraint.model(p) for p in param_samples]))

 41%|█████████████████████████▏                                    | 65/160 [01:35<03:15,  2.06s/it]

## Write posterior predictive distributions to disk

In [ ]:
with open(output_dir / "nn_elm_prior_predictive.pkl", "wb") as f:
    pickle.dump(yth_nn, f)

In [ ]:
with open(output_dir / "pp_elm_prior_predictive.pkl", "wb") as f:
    pickle.dump(yth_pp, f)